<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using LangChain's SQLDatabaseToolkit with SingleStoreDB</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!pip install langchain langchain-community langchain-openai --quiet

In [6]:
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used to make connections to the selected database.</p>
    </div>
</div>

In [8]:
db = SQLDatabase.from_uri(connection_url, include_tables = ["customer", "nation"])

llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0, verbose = False)

toolkit = SQLDatabaseToolkit(db = db, llm = llm)

agent_executor = create_sql_agent(
    llm = llm,
    toolkit = toolkit,
    max_iterations = 15,
    max_execution_time = 60,
    top_k = 3,
    verbose = False
)

In [9]:
error_string = "Could not parse LLM output:"

def run_agent_query(query, agent_executor, error_string):
    try:
        result = agent_executor.invoke(query, return_only_outputs = True)["output"]
    except Exception as e:
        error_message = str(e)
        # Check if the error message contains the specific string
        if error_string in error_message:
            # Extract the part after the specific string and strip backticks
            result = error_message.split(error_string)[1].strip().strip('`')
        else:
            result = f"Error occurred: {error_message}"
    return result

In [10]:
query = (
    "Using the customer and nation tables, write a SingleStore query\n"
    "that shows the highest paying customers per country,\n"
    "include how much they've spent, use the nation name."
)

result = run_agent_query(query, agent_executor, error_string)
print(result)

The highest paying customers per country are:
1. **Customer**: Customer#000508503 from **Country**: UNITED STATES with an amount spent of 9999.99.
2. **Customer**: Customer#000061453 from **Country**: MOROCCO with an amount spent of 9999.99.
3. **Customer**: Customer#001123705 from **Country**: MOZAMBIQUE with an amount spent of 9999.98.
